In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ..

/home/iovcharenko/Documents/NotWork/UCU/liner-algebra/ucu-linear-algebra-final-project


In [65]:
import numpy as np
import pandas as pd

from pathlib import Path

from src.metrics import rmse
from src.utils import movies_info_preproc

In [4]:
data_folder = Path("data")
results_folder = Path("results")
figs_folder = Path("figs")

In [104]:
df = pd.read_csv(data_folder / "subsets" / "low-sparsity" / "records.csv")
tr_df = df[df.split == "train"].copy()
val_df = df[df.split == "val"].copy()

movies_df = pd.read_csv(data_folder / "subsets" / "low-sparsity" / "movies.csv")
df = pd.merge(
    df,
    movies_df,
    left_on="movie_id",
    right_on="subset_id"
).drop(
    columns="subset_id"
)


movies_titles_df = pd.read_csv(
    data_folder / "meta" / "movie_titles.csv", 
    names=["movie_id", "release_date", "movie_name"],
    encoding="iso-8859-1"
)
df = pd.merge(
    df,
    movies_titles_df,
    left_on="netflix_id",
    right_on="movie_id",
    suffixes=("", "_copy")
).drop(
    columns="movie_id_copy"
)


movies_info_df = pd.read_csv(data_folder / "meta" / "movie_info.csv")
movies_info_df = movies_info_preproc(movies_info_df)
movies_info_df = pd.merge(
    movies_info_df,
    df.groupby(["movie_name", "movie_id"]) \
      .size().reset_index().drop(columns=0),
    on="movie_name",
)


In [116]:
def df_with_meta(df):
    return df[df.movie_id.isin(movies_info_df.movie_id.unique())].reset_index(drop=True)

In [6]:
results_df = pd.read_csv(results_folder / "15-02-21" / "val_preds.csv")

In [10]:
results_df

,movie_id,customer_id,rating,date,split,als-svd,funk-svd,nnmf,neural-svd,surprise-svd
0,540,646,3.0,1999-12-31,val,2.880528,2.599780,3.658644,3.270864,2.877070
1,723,646,2.0,2000-01-06,val,3.271187,3.434981,3.869058,3.270864,3.480361
2,398,509,4.0,2000-01-06,val,3.882678,4.064758,4.153707,3.762923,4.039360
3,391,509,4.0,2000-01-06,val,3.999982,4.081499,4.102665,3.762923,4.003301
4,412,646,1.0,2000-01-06,val,2.908660,3.536602,3.888122,3.270864,3.700313
...,...,...,...,...,...,...,...,...,...,...
28947,436,668,3.0,2005-12-31,val,4.178313,3.799638,4.103028,3.895321,4.276958
28948,410,668,3.0,2005-12-31,val,3.113027,3.335705,3.780240,3.895321,3.367744
28949,840,668,3.0,2005-12-31,val,3.492049,3.550631,3.874934,3.895321,3.581163
28950,347,951,4.0,2005-12-31,val,4.255986,4.366113,3.882686,4.055395,4.571869


In [29]:
algorithms = ['als-svd', 'funk-svd', 'nnmf', 'neural-svd', 'surprise-svd']

for algo_name in algorithms:
    score_per_user = results_df \
                    .groupby("customer_id") \
                    .apply(lambda df: rmse(df.rating, df[algo_name])) \
                    .sort_values() \
                    .reset_index().rename(columns={0: "rmse_score"})
            
    best_user = score_per_user.iloc[0]
    worst_user = score_per_user.iloc[-1]
    
    break 

In [30]:
best_user

customer_id    883.00000
rmse_score       0.37675
Name: 0, dtype: float64

In [130]:
def get_user_top_movies(user, k, df, rating_field="rating"):
    user_df = df[df.customer_id == user.customer_id].sort_values([rating_field, "date"])
    result_columns = ["movie_id", rating_field]
    return (
        user_df.iloc[:k][result_columns].reset_index(drop=True),
        user_df.iloc[-k:][result_columns].reset_index(drop=True),
    )

a, b = get_user_top_movies(best_user, 3, df_with_meta(results_df), "als-svd")

In [127]:
pd.merge(a, movies_info_df, on="movie_id", how="left")

,movie_id,als-svd,movie_name,genre,overview,poster_url,language,alternative_title
0,540,2.880528,Home Alone 2: Lost in New York,"Comedy, Family, Adventure","Instead of flying to Florida with his folks, K...",https://image.tmdb.org/t/p/original/uuitWHpJwx...,en,Один дома 2: Затерянный в Нью-Йорке
1,723,3.271187,Muriel's Wedding,"Drama, Comedy, Romance",A young social outcast in Australia steals mon...,https://image.tmdb.org/t/p/original/zJyTr8Fo41...,en,None
2,398,3.882678,Lethal Weapon,"Adventure, Action, Comedy, Thriller, Crime",Veteran buttoned-down LAPD detective Roger Mur...,https://image.tmdb.org/t/p/original/fTq4ThIP3p...,en,None
